<a href="https://colab.research.google.com/github/easthometown/testrep/blob/main/C1_W5_Lab_1_exploring_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [ ]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [ ]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [ ]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [ ]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [ ]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [ ]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [ ]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [ ]:
!rm -rf logs

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 19s 749ms/step - loss: 0.6819 - accuracy: 0.5742 - val_loss: 0.6384 - val_accuracy: 0.7317
Epoch 2/10
26/26 [==============================] - 19s 715ms/step - loss: 0.6042 - accuracy: 0.6861 - val_loss: 0.7497 - val_accuracy: 0.5073
Epoch 3/10
26/26 [==============================] - 18s 700ms/step - loss: 0.5697 - accuracy: 0.7178 - val_loss: 0.5586 - val_accuracy: 0.7268
Epoch 4/10
26/26 [==============================] - 18s 704ms/step - loss: 0.5364 - accuracy: 0.7652 - val_loss: 0.4749 - val_accuracy: 0.8244
Epoch 5/10
26/26 [==============================] - 18s 696ms/step - loss: 0.4535 - accuracy: 0.8005 - val_loss: 0.4920 - val_accuracy: 0.7561
Epoch 6/10
26/26 [==============================] - 18s 700ms/step - loss: 0.3956 - accuracy: 0.8491 - val_loss: 0.4278 - val_accuracy: 0.8146
Epoch 7/10
26/26 [==============================] - 18s 704ms/step - loss: 0.3664 - accuracy: 0.8467 - val_loss: 0.3013 - val_accuracy: 0.8927

In [ ]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.64.h5
26/26 - 19s - loss: 0.6734 - accuracy: 0.5815 - val_loss: 0.6354 - val_accuracy: 0.7805
Epoch 2/5

Epoch 00002: saving model to weights.02-0.57.h5
26/26 - 18s - loss: 0.6084 - accuracy: 0.6922 - val_loss: 0.5706 - val_accuracy: 0.7366
Epoch 3/5

Epoch 00003: saving model to weights.03-0.61.h5
26/26 - 19s - loss: 0.5640 - accuracy: 0.7190 - val_loss: 0.6063 - val_accuracy: 0.6098
Epoch 4/5

Epoch 00004: saving model to weights.04-0.43.h5
26/26 - 19s - loss: 0.4910 - accuracy: 0.7944 - val_loss: 0.4349 - val_accuracy: 0.8683
Epoch 5/5

Epoch 00005: saving model to weights.05-0.41.h5
26/26 - 19s - loss: 0.4868 - accuracy: 0.7859 - val_loss: 0.4112 - val_accuracy: 0.8927


In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 19s - loss: 0.6774 - accuracy: 0.5900 - val_loss: 0.6617 - val_accuracy: 0.6634


In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 19s - loss: 0.6704 - accuracy: 0.6119 - val_loss: 0.6713 - val_accuracy: 0.4585
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 19s - loss: 0.6307 - accuracy: 0.6813 - val_loss: 0.5999 - val_accuracy: 0.7707


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 21s - loss: 0.6645 - accuracy: 0.5852 - val_loss: 0.6359 - val_accuracy: 0.7659
Epoch 2/50
26/26 - 19s - loss: 0.6247 - accuracy: 0.6302 - val_loss: 0.6047 - val_accuracy: 0.7512
Epoch 3/50
26/26 - 18s - loss: 0.5785 - accuracy: 0.7263 - val_loss: 0.5260 - val_accuracy: 0.8146
Epoch 4/50
26/26 - 18s - loss: 0.5156 - accuracy: 0.7835 - val_loss: 0.5211 - val_accuracy: 0.7756
Epoch 5/50
26/26 - 20s - loss: 0.4623 - accuracy: 0.7968 - val_loss: 0.3839 - val_accuracy: 0.8927
Epoch 6/50
26/26 - 18s - loss: 0.4361 - accuracy: 0.8333 - val_loss: 0.3267 - val_accuracy: 0.9024
Epoch 7/50
26/26 - 18s - loss: 0.3304 - accuracy: 0.8978 - val_loss: 0.3618 - val_accuracy: 0.8780
Epoch 8/50
26/26 - 19s - loss: 0.2689 - accuracy: 0.8942 - val_loss: 0.6732 - val_accuracy: 0.6293
Epoch 9/50
26/26 - 19s - loss: 0.2339 - accuracy: 0.9282 - val_loss: 0.1624 - val_accuracy: 0.9707
Epoch 10/50
26/26 - 19s - loss: 0.1562 - accuracy: 0.9708 - val_loss: 0.2092 - val_accuracy: 0.9268
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 19s 737ms/step - loss: 0.6775 - accuracy: 0.5706 - val_loss: 0.6691 - val_accuracy: 0.4927
Epoch 2/5
26/26 [==============================] - 19s 732ms/step - loss: 0.6189 - accuracy: 0.6618 - val_loss: 0.5838 - val_accuracy: 0.7951
Epoch 3/5
26/26 [==============================] - 19s 719ms/step - loss: 0.5693 - accuracy: 0.7384 - val_loss: 0.7790 - val_accuracy: 0.4878
Epoch 4/5
26/26 [==============================] - 19s 712ms/step - loss: 0.5098 - accuracy: 0.7713 - val_loss: 0.5045 - val_accuracy: 0.7463
Epoch 5/5
26/26 [==============================] - 18s 708ms/step - loss: 0.4916 - accuracy: 0.7908 - val_loss: 0.4098 - val_accuracy: 0.9268


In [ ]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.570560,0.678008,0.492683,0.669134
1,1,0.661801,0.619011,0.795122,0.583790
2,2,0.738443,0.570522,0.487805,0.779035
3,3,0.771290,0.510143,0.746341,0.504480
4,4,0.790754,0.492236,0.926829,0.409797


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 19s 745ms/step - loss: 0.6760 - accuracy: 0.5742 - val_loss: 0.6572 - val_accuracy: 0.7171

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 41s 2s/step - loss: 0.6357 - accuracy: 0.6825 - val_loss: 0.6449 - val_accuracy: 0.6293

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 28s 1s/step - loss: 0.6175 - accuracy: 0.7372 - val_loss: 0.6248 - val_accuracy: 0.7902

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 28s 1s/step - loss: 0.6071 - accuracy: 0.7652 - val_loss: 0.6253 - val_accuracy: 0.7171

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 28s 1s/step - loss: 0.6012 - accura

In [ ]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 33s 1s/step - loss: 0.6752 - accuracy: 0.5742 - val_loss: 0.7116 - val_accuracy: 0.4585
Epoch 2/50
26/26 [==============================] - 21s 799ms/step - loss: 0.6268 - accuracy: 0.6350 - val_loss: 0.6969 - val_accuracy: 0.4976
Epoch 3/50
26/26 [==============================] - 21s 812ms/step - loss: 0.5893 - accuracy: 0.7129 - val_loss: 0.5591 - val_accuracy: 0.7951
Epoch 4/50
26/26 [==============================] - 20s 780ms/step - loss: 0.5464 - accuracy: 0.7543 - val_loss: 0.5005 - val_accuracy: 0.7805
Epoch 5/50
26/26 [==============================] - 21s 796ms/step - loss: 0.5134 - accuracy: 0.7737 - val_loss: 0.4374 - val_accuracy: 0.8927
Epoch 6/50
26/26 [==============================] - 21s 796ms/step - loss: 0.4316 - accuracy: 0.8443 - val_loss: 0.3710 - val_accuracy: 0.9024
Epoch 7/50
26/26 [==============================] - 23s 881ms/step - loss: 0.4037 - accuracy: 0.8467 - val_loss: 0.3415 - val_accuracy: 0.8976
Ep

In [ ]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4488), started 0:16:09 ago. (Use '!kill 4488' to kill it.)